In [1]:
import tflearn

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [3]:
import cv2
import numpy as np
import os
from random import shuffle
import glob

In [4]:
train_dir1="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Apple Braeburn"
train_dir2="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Apple Golden 1"
train_dir3="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Apple Golden 2"
train_dir4="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Apple Golden 3"
train_dir5="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Apple Granny Smith"
train_dir6="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Apple Red 1"
train_dir7="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Apple Red 2"
train_dir8="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Banana"
train_dir9="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Banana Red"
train_dir10="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Dates"
train_dir11="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Pineapple"
train_dir12="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Raspberry"
test_dir= "/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Validation/Apple Red 1"

In [5]:
a=[train_dir1,train_dir2,train_dir3,train_dir4,train_dir5,train_dir6,train_dir7]


In [6]:
training_data=[]
for i in a:
    filenames=glob.glob(i+"/*.jpg")
    IMG_SIZE=80
    for f in filenames:
        img=cv2.imread(f)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE)) #resizing image
        normalizedImg = np.zeros((IMG_SIZE,IMG_SIZE))
        normalizedImg = cv2.normalize(img,  normalizedImg, 0,255, cv2.NORM_MINMAX)#normalizing and scaling
        img = cv2.cvtColor( normalizedImg, cv2.COLOR_RGB2GRAY )
        training_data.append([np.array(img),np.array([1,0])])
    
    
    


    

In [7]:
b=[train_dir8,train_dir9,train_dir10,train_dir11,train_dir12]
for i in b:
    filenames=glob.glob(i+"/*.jpg")
    IMG_SIZE=80
    for f in filenames:
        img=cv2.imread(f)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE)) #resizing image
        normalizedImg = np.zeros((IMG_SIZE,IMG_SIZE))
        normalizedImg = cv2.normalize(img,  normalizedImg, 0,255, cv2.NORM_MINMAX)#normalizing and scaling
        img = cv2.cvtColor( normalizedImg, cv2.COLOR_RGB2GRAY )
        training_data.append([np.array(img),np.array([0,1])])
    

In [8]:
shuffle(training_data) #shuffling training data so that same distribution 

In [9]:
filenames=glob.glob(test_dir+"/*.jpg")
test_data=[]
IMG_SIZE=80
for f in filenames:
    img=cv2.imread(f)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE)) #resizing image
    normalizedImg = np.zeros((IMG_SIZE,IMG_SIZE))
    normalizedImg = cv2.normalize(img,  normalizedImg, 0,255, cv2.NORM_MINMAX)#normalizing and scaling
    img = cv2.cvtColor( normalizedImg, cv2.COLOR_RGB2GRAY )
    test_data.append([np.array(img),np.array([1,0])])

In [10]:
import tensorflow as tf
tf.reset_default_graph()

In [13]:
LR = 1e-3
MODEL_NAME='apple detector1'
import tensorflow as tf
tf.reset_default_graph()

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

train = training_data
test = test_data

X = np.array([i[0] for i in training_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in training_data]

test_x = np.array([i[0] for i in test_data]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test_data]

model.fit({'input': X}, {'targets': Y}, n_epoch=2, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True)



Training Step: 183  | total loss: 0.00008 | time: 180.699s
| Adam | epoch: 002 | loss: 0.00008 - acc: 1.0000 -- iter: 5824/5883
Training Step: 184  | total loss: 0.00007 | time: 184.739s
| Adam | epoch: 002 | loss: 0.00007 - acc: 1.0000 | val_loss: 0.00000 - val_acc: 1.0000 -- iter: 5883/5883
--


In [77]:
image=cv2.imread("/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Validation/Banana/102_100.jpg")
img = cv2.resize(image, (IMG_SIZE,IMG_SIZE))
normalizedImg = np.zeros((IMG_SIZE,IMG_SIZE))
normalizedImg = cv2.normalize(img,  normalizedImg, 0,255, cv2.NORM_MINMAX)#normalizing and scaling
img = cv2.cvtColor( normalizedImg, cv2.COLOR_RGB2GRAY )

In [78]:
a=model.predict(img.reshape(-1,IMG_SIZE,IMG_SIZE,1))


In [79]:
if(a[0][0]>a[0][1]):
    print('yes,this is apple')
else:
    print('no,not a apple')


no,not a apple


In [81]:
#To covert data into tfrecord file
from PIL import Image
import numpy as np
import skimage.io as io
import tensorflow as tf
import glob


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

tfrecords_filename = 'pascal_voc_segmentation.tfrecords'

writer = tf.python_io.TFRecordWriter(tfrecords_filename)


original_images = []
train_dir1="/home/spartan/Desktop/RedCarpet_Project/Fruit-Images-Dataset-master/Training/Apple Braeburn"
filenames=glob.glob(train_dir1+"/*.jpg")

for img_path in filenames:
    
    img = np.array(Image.open(img_path))
   
    
    
    height = img.shape[0]
    width = img.shape[1]
    
    
    original_images.append((img))
    
    img_raw = img.tostring()
    
    
    example = tf.train.Example(features=tf.train.Features(feature={
        'height': _int64_feature(height),
        'width': _int64_feature(width),
        'image_raw': _bytes_feature(img_raw),
        }))
    
    writer.write(example.SerializeToString())

writer.close()